In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve

In [4]:
original = pd.read_csv("../../data/processed/original (5).csv")

# How can we predict and mitigate customer churn using machine learning techniques?

## Develop a machine learning model to predict the likelihood of customer churn.

In [8]:
original['Churn'] = original['Attrition_Flag'].apply(lambda x: 'No' if x == 'Existing Customer' else 'Yes')
ori = original.copy()
ori = original.drop(columns = ['Attrition_Flag'])

In [10]:
total_customers = ori['Churn'].count()
churners = ori[ori['Churn'] == 'Yes'].shape[0]
churn_rate = churners / total_customers * 100
print(f"The Churn Rate is quite high at {churn_rate:.2f}%.")

The Churn Rate is quite high at 15.72%.


### Likelihood of Churning for each customer

#### Preparing the data

In [37]:
np.random.seed(10)

# encoding 'Income_Category', 'Product_Usage', 'Cluster_Labels' and 'Churn' to numerical formats
encoders = {}
for c in ['Gender', 'Income_Category', 'Education_Level', 'Marital_Status', 'No_of_product', 'Card_Category'
          , 'Churn']:
    le = LabelEncoder()
    ori[c] = le.fit_transform(ori[c])
    encoders[c] = le

# splitting the data into training and test
## training data size: 80%; test data size: 20%
X = ori.drop(columns = ['Churn'])
y = ori['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

#### Random Forest Classifier

To add Churn_Probability to the data.

In [40]:
model = RandomForestClassifier(random_state = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
churn_probabilities = model.predict_proba(X)[:, 1]

# add likelihood of a customer churning into the dataset
original['Churn_Probability'] = churn_probabilities

#### Testing viability

##### Accuracy and F1 scores

In [43]:
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 score: {f1:.4f}")

Accuracy: 0.9318
F1 score: 0.7530


The accuracy is quite high at 93.18%. This shows that the model accurately classifies the customer into churned or not churned 94.27% of the time. However, this should not be trusted because most people did not churn (83.93%) in the data.

The F1 score is relatively high 75.30%. It is average of precision and recall and both of these contribute equally to the F1 score (Sharma, 2023). The model as a modest ability in predicting true positives (customers who actually churn). 

##### AUC - ROC 

AUC-ROC score: 0.9684


In [49]:
y_prob = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print(f"AUC-ROC score: {auc:.4f}")

# plotting the AUC-ROC
false_pos_r, true_pos_r, thresholds = roc_curve(y_test, y_prob)
plt.plot(false_pos_r, true_pos_r)
plt.plot([0, 1], [0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

The AUC-ROC score is high at 96.84%. The model has significant in predicting the likelihood of a customer churning. The model correctly classifies with a probability of 96.84%.

### Early Warning System

In [53]:
# Calculate mean and median Transaction_Behavior for churned vs. non-churned customers
churned = ori[ori['Churn'] == 1]
non_churned = ori[ori['Churn'] == 0]

churned_mean = churned['Transaction_Behavior'].mean()
non_churned_mean = non_churned['Transaction_Behavior'].mean() 

print("Average Transaction_Behavior for churned customers:", churned_mean)
print("Average Transaction_Behavior for non-churned customers:", non_churned_mean)
# Since churned customers have a lower Transaction_Behavior value on average, it suggests that lower Transaction_Behavior values correlate with a higher churn risk.

# Defining threshold criteria for high-risk customers
cluster_risk_threshold = [1, 3]  # Clusters likely associated with higher churn risk
churn_status_threshold = 1  # Let 1 = churned, 0 = non-churned
transaction_behavior_threshold = 1.3  # Threshold for frequent transactions
product_usage_threshold = 1  # Flagging customers with a high product usage

# Apply early warning system criteria
ori['Alert'] = ori.apply(
    lambda row: 'High risk of churn - follow up needed' if (
        row['Cluster_Labels'] in cluster_risk_threshold or  # High-risk cluster
        row['Churn'] >= churn_status_threshold or           # Churned or likely to churn
        row['Transaction_Behavior'] < transaction_behavior_threshold or  # High transactions
        row['Product_Usage'] > product_usage_threshold      # High product usage
    ) else 'Low risk',
    axis=1
)

# Display results
print(ori[['CLIENTNUM', 'Churn', 'Cluster_Labels', 'Transaction_Behavior', 'Product_Usage', 'Alert']])

# Count high-risk and low-risk customers
risk_counts = ori['Alert'].value_counts()
print("\nCustomer Risk Summary:")
print(risk_counts) # high risk = 3760, low risk = 3321

KeyError: 'Transaction_Behavior'

#### Retention Strategies

1. Personalized Communication Targeted Messaging: Use customer data to send tailored emails that acknowledge individual preferences and behaviors. Highlight relevant promotions or services to create a more personalized experience, which can significantly enhance engagement and satisfaction.

2.  Incentives and Rewards Loyalty Programs: Implement a rewards program that encourages continued use of your services. Offering points, discounts, or exclusive benefits can motivate high-risk customers to remain loyal and feel valued.

3. Enhanced Customer Support Dedicated Account Management: Assign dedicated account managers to high-risk customers. This personalized support ensures their concerns are addressed promptly and reinforces the commitment to customer satisfaction, fostering a stronger relationship and trust.

In [59]:
# for API
A_BQ3 = original[["CLIENTNUM", "Churn_Probability"]]
A_BQ3.to_csv('../data/processed/temp4.csv', index = False)